Câu 1:

a,lập trình hướng đối tượng oop

In [1]:
class NhanVien:
    def __init__(self, ma_nv, luong_cb):
        self.__ma_nv = ma_nv       
        self.__luong_cb = luong_cb  

    def tinh_luong(self):
        return self.__luong_cb

    def get_ma_nv(self):
        return self.__ma_nv

    def get_luong_cb(self):
        return self.__luong_cb


In [2]:
class NhanVienTangCa(NhanVien):
    def tinh_luong(self):
        return self.get_luong_cb() * 1.2


In [3]:
class NhanVienChuyenChuyen(NhanVien):
    def tinh_luong(self):
        return self.get_luong_cb() + 500000


In [ ]:

ds_nhan_vien = [
    NhanVien("NV01", 10_000_000),
    NhanVienTangCa("NV02", 10_000_000),
    NhanVienChuyenChuyen("NV03", 10_000_000)
]

# Duyệt danh sách và in kết quả
for nv in ds_nhan_vien:
    print(f"Mã NV: {nv.get_ma_nv()} - Lương: {nv.tinh_luong():,.0f} VND")


Mã NV: NV01 - Lương: 10,000,000 VND
Mã NV: NV02 - Lương: 12,000,000 VND
Mã NV: NV03 - Lương: 10,500,000 VND


b, NUMPY & XỬ lý bảng

In [5]:
import numpy as np

san_luong = [12, None, 8, 15, None, 10]

arr = np.array(san_luong, dtype=float)
arr


array([12., nan,  8., 15., nan, 10.])

In [ ]:
#thực hiện reshape thành mảng kích thước 2x
arr_reshape = arr.reshape(2, 3)
arr_reshape

array([[12., nan,  8.],
       [15., nan, 10.]])

In [7]:
# Tính giá trị trung bình (bỏ qua NaN)
mean_value = np.nanmean(arr_reshape)

# Thay NaN bằng mean
arr_filled = np.nan_to_num(arr_reshape, nan=mean_value)
arr_filled

array([[12.  , 11.25,  8.  ],
       [15.  , 11.25, 10.  ]])

In [8]:
# trích xuât phần từ >9
lon_hon_9 = arr_filled[arr_filled > 9]
lon_hon_9

array([12.  , 11.25, 15.  , 11.25, 10.  ])

In [ ]:
# giá trị nhỏ nhất trog tập hợp phần tử lớn hơn 9 vừa trích xuất dc 
min_value = np.min(lon_hon_9)
min_value


np.float64(10.0)

Câu 2: xử lý dữ liệu thực tế với pandas và sql

a, lọc dữ liệu

In [2]:
import pandas as pd
import numpy as np
import sqlite3

# Đọc dữ liệu
df = pd.read_csv("data/nhan_su.csv")
df

df_loc = df[
    df["Ma_NV"]
    .astype(str)                     # xử lý giá trị thiếu
    .str.contains("NS", case=False, na=False)
    &
    (df["Nam_Cong_Tac"] >= 2021)
]

df_loc


,Ma_NV,Phong_Ban,Thanh_Tich,Nam_Cong_Tac
0,NS001,KinhDoanh,85.0,2021
1,ns002,KinhDoanh,NaN,2022
2,NS003,NhanSu,90.0,2023
4,NS005,KyThuat,88.0,2021
5,ns006,KyThuat,NaN,2024
6,NS007,KinhDoanh,95.0,2023
8,NS009,NhanSu,NaN,2022
9,NS010,KyThuat,92.0,2024
10,NS011,KinhDoanh,80.0,2021


b, Làm sạch dữ liệu cột thành tích bằng gtri tbinh của chính cột 

In [3]:
mean_thanh_tich = df_loc["Thanh_Tich"].mean()

df_loc["Thanh_Tich"] = df_loc["Thanh_Tich"].fillna(mean_thanh_tich)

df_loc


C:\Users\Admin\AppData\Local\Temp\ipykernel_4372\4265981987.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc["Thanh_Tich"] = df_loc["Thanh_Tich"].fillna(mean_thanh_tich)


,Ma_NV,Phong_Ban,Thanh_Tich,Nam_Cong_Tac
0,NS001,KinhDoanh,85.000000,2021
1,ns002,KinhDoanh,88.333333,2022
2,NS003,NhanSu,90.000000,2023
4,NS005,KyThuat,88.000000,2021
5,ns006,KyThuat,88.333333,2024
6,NS007,KinhDoanh,95.000000,2023
8,NS009,NhanSu,88.333333,2022
9,NS010,KyThuat,92.000000,2024
10,NS011,KinhDoanh,80.000000,2021


c, tổng hợp dữ liệu groupby để tính tổng điểm thành tích theo từng phòng 

In [4]:
tong_thanh_tich = df_loc.groupby("Phong_Ban")["Thanh_Tich"].sum()

tong_thanh_tich


Phong_Ban
KinhDoanh    348.333333
KyThuat      268.333333
NhanSu       178.333333
Name: Thanh_Tich, dtype: float64

d, Tạo bảng Pivot table 

In [5]:
pivot_table = pd.pivot_table(
    df_loc,
    index="Phong_Ban",
    columns="Nam_Cong_Tac",
    values="Thanh_Tich",
    aggfunc="sum",
    fill_value=0
)

pivot_table


Nam_Cong_Tac,2021,2022,2023,2024
Phong_Ban,,,,
KinhDoanh,165.0,88.333333,95.0,0.000000
KyThuat,88.0,0.000000,0.0,180.333333
NhanSu,0.0,88.333333,90.0,0.000000


e, lưu trữ và truy vấn dữ liệu sdung sql

In [27]:
# Kết nối SQLite
conn = sqlite3.connect("OUTPUT/HeThong.db")

# Lưu bảng
pivot_table.to_sql(
    "bang_luong_2024",
    conn,
    if_exists="replace"
)


3

In [28]:
# thực hiện truy vấn 5 dòng đầu tiên để kiểm tra 
query = "SELECT * FROM bang_luong_2024 LIMIT 5;"
df_check = pd.read_sql(query, conn)

df_check


,Phong_Ban,2021,2022,2023,2024
0,KinhDoanh,165.0,88.333333,95.0,0.000000
1,KyThuat,88.0,0.000000,0.0,180.333333
2,NhanSu,0.0,88.333333,90.0,0.000000
